In [5]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
import json
from selenium import webdriver

## Download the configs i.e. phone # and password

In [6]:
with open("personal_config.json") as f:
    configs = json.load(f)
login_url = "https://www.lycamobile.us/en/"

## create low head webdriver to scrape the data
 - requests library cannot be used because when we scrape the page, data is loaded dynamically using Ajax calls and Javascript

In [7]:
# download phantomjs driver
# other drivers available
browser = webdriver.PhantomJS(executable_path="/Users/srossg/Downloads/phantomjs-2.1.1-macosx/bin/phantomjs")
browser.set_window_size(1124, 850)


/usr/local/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


## Enter login page 

In [8]:
browser.get(login_url)
button  = browser.find_element_by_id("logginedUserButton")
button.click()


In [9]:
button  = browser.find_element_by_id("login_link")
button.click()

In [10]:
tel_no = browser.find_element_by_id("lyca_login_mobile_no")
password = browser.find_element_by_id("lyca_login_password")

## enter mobile# and password

In [11]:
tel_no.send_keys(configs["tel"])
password.send_keys(configs["password"])

In [12]:
button  = browser.find_element_by_id("login_submit")
button.click()

In [13]:
output = browser.find_elements_by_id("result-set")

## Attach another session to same browser session

In [14]:
url = browser.command_executor._url       #"http://127.0.0.1:60622/hub"
session_id = browser.session_id

In [15]:
driver = webdriver.Remote(command_executor=url,desired_capabilities={})
driver.session_id = session_id

## get all the months data available 

In [16]:
driver.get("https://www.lycamobile.us/en/my-account/callhistory/")
data1={"call_type":[],"dialled_no":[],"date":[],"time":[],"duration":[],"cost":[]}
li=[]
month_select = Select(driver.find_element_by_id("month-year"))

for each in month_select.options:
    li.append(each.text.strip())
li.remove('Select Month')

In [23]:
li

['May-2019',
 'April-2019',
 'March-2019',
 'February-2019',
 'January-2019',
 'December-2018']

In [44]:
month_select = Select(driver.find_element_by_id("month-year"))
month_select.select_by_value(li[4])

In [45]:
total_pagination = driver.find_element(by="className",value='numpagescount')
total_pagination_value = int(total_pagination.text.split()[-1])
total_pagination_value

39

In [47]:
for i in range(1,total_pagination_value):
    table = driver.find_element_by_xpath("//table[@class='table-for-call-history']")
    for table_row in table.find_elements_by_xpath(".//tr"):
        items_val = table_row.find_elements_by_xpath(".//td")
        if len(items_val)>0:
            data1["call_type"].append(items_val[0].text)
            data1["dialled_no"].append(items_val[1].text)
            data1["date"].append(items_val[2].text)
            data1["time"].append(items_val[3].text)
            data1["duration"].append(items_val[4].text)
            data1["cost"].append(items_val[5].text)
    button  = driver.find_element_by_id("nextValue")
    button.click()
    time.sleep(1)

## Automated the above process. 
## TODO: Just need to put appropriate sleep value to fetch correct pagination value

In [ ]:
for month in li:
    month_select.select_by_value(month)
    time.sleep(2)
    total_pagination = driver.find_element(by="className",value='numpagescount')
    total_pagination_value = int(total_pagination.text.split()[-1])
    print(month,total_pagination_value)
    for i in range(1,total_pagination_value):
        table = driver.find_element_by_xpath("//table[@class='table-for-call-history']")
        for table_row in table.find_elements_by_xpath(".//tr"):
            items_val = table_row.find_elements_by_xpath(".//td")
            if len(items_val)>0:
                data1["call_type"].append(items_val[0].text)
                data1["dialled_no"].append(items_val[1].text)
                data1["date"].append(items_val[2].text)
                data1["time"].append(items_val[3].text)
                data1["duration"].append(items_val[4].text)
                data1["cost"].append(items_val[5].text)
        button  = driver.find_element_by_id("nextValue")
        button.click()
        time.sleep(1)

In [48]:
df = pd.DataFrame(data1)

In [50]:
df.to_csv("lycalogs.csv")

In [ ]:
driver.save_screenshot('screenshot.png')

In [ ]:
browser.save_screenshot('screenshot.png')